In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [3]:
df=pd.read_csv('car_sales_data.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Manufacturer         49991 non-null  object 
 1   Model                49988 non-null  object 
 2   Engine size          49989 non-null  float64
 3   Fuel type            49988 non-null  object 
 4   Year of manufacture  49975 non-null  float64
 5   Mileage              49986 non-null  float64
 6   Price                49985 non-null  float64
dtypes: float64(4), object(3)
memory usage: 2.7+ MB


In [5]:
df.nunique()

Manufacturer               5
Model                     15
Engine size               14
Fuel type                  3
Year of manufacture       39
Mileage                44959
Price                  25040
dtype: int64

In [14]:
#missing valueni toldirish
for col in df.columns:
    if df[col].isnull().any():
        if df[col].dtype=='object':
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:
            df[col].fillna(df[col].mean(), inplace=True)
            

In [15]:
for col in df.columns.drop('Model'):
    if df[col].dtype == 'object':
        if df[col].nunique() <= 15:
            dummies = pd.get_dummies(df[col], prefix=col, dtype=int)
            df = pd.concat([df.drop(columns=col), dummies], axis=1)
        else:
            df[col] = encoder.fit_transform(df[col])
            

In [19]:
#scaling
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model                 50000 non-null  object 
 1   Engine size           50000 non-null  float64
 2   Year of manufacture   50000 non-null  float64
 3   Mileage               50000 non-null  float64
 4   Price                 50000 non-null  float64
 5   Manufacturer_BMW      50000 non-null  int64  
 6   Manufacturer_Ford     50000 non-null  int64  
 7   Manufacturer_Porsche  50000 non-null  int64  
 8   Manufacturer_Toyota   50000 non-null  int64  
 9   Manufacturer_VW       50000 non-null  int64  
 10  Fuel type_Diesel      50000 non-null  int64  
 11  Fuel type_Hybrid      50000 non-null  int64  
 12  Fuel type_Petrol      50000 non-null  int64  
dtypes: float64(4), int64(8), object(1)
memory usage: 5.0+ MB


MODELLING


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
model=LogisticRegression()
x=df.drop('Model', axis=1)
y=df['Model']

In [21]:
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.2, random_state=42)


In [22]:
x_train.shape, x_test.shape


((40000, 12), (10000, 12))

In [23]:
y_train.shape, y_test.shape

((40000,), (10000,))

In [24]:
model.fit(x_train,y_train)

C:\Users\maxma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
y_pred=model.predict(x_test)

In [26]:
y_pred


array(['RAV4', 'Focus', 'Mondeo', ..., 'Mondeo', 'Passat', 'Passat'],
      shape=(10000,), dtype=object)

In [27]:
from sklearn.metrics import accuracy_score, classification_report
score=accuracy_score(y_test,y_pred)
report=classification_report(y_test,y_pred)


In [28]:
score

0.7035

In [29]:
print(report)

              precision    recall  f1-score   support

  718 Cayman       0.42      0.59      0.49       165
         911       0.64      0.56      0.60       188
     Cayenne       0.60      0.47      0.52       182
      Fiesta       0.75      0.90      0.82       933
       Focus       0.76      0.45      0.56      1002
        Golf       0.54      0.28      0.37      1074
          M5       1.00      1.00      1.00       302
      Mondeo       0.72      0.88      0.79      1005
      Passat       0.66      0.80      0.72       953
        Polo       0.56      0.73      0.64       966
       Prius       0.59      0.73      0.65       848
        RAV4       0.86      0.89      0.87       857
          X3       1.00      0.77      0.87       352
       Yaris       0.83      0.60      0.70       831
          Z4       0.81      1.00      0.89       342

    accuracy                           0.70     10000
   macro avg       0.72      0.71      0.70     10000
weighted avg       0.71   